# Flujo de ventas de juegos


## Se van a escoger 700 usuarios de los 800 existentes para que compren entre 1-5 juegos en diferentes fechas

In [ ]:
import random
import mysql.connector
import json
import string
from pymongo import MongoClient
import datetime as dt

def random_string(num):
    return ''.join([random.choice(string.ascii_letters + string.digits) for n in range(num)]) + ' '

def random_text():
    num = random.randrange(1,15)
    num2 = random.randrange(1,200)
    ret = ''
    for i in range(0, num2) :
        ret.join(random_string(num))
    return ret
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["usuarios"]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)

mycursor = mydb.cursor()



for i in range(1,701):
    #vamos a ver cuantos juegos quiere comprar el usuario
    numjue = random.randrange(2,6)
    for a in range(1,numjue):
        #Procedemos a comprobar que claves tenemos no vendidas
        mycursor.execute("SELECT c.id_clave FROM claves_juegos c WHERE vendido = 0")
        claves_libres = mycursor.fetchall()
        id = random.randrange(0,len(claves_libres))
        #sacamos una clave de entre las no vendidas
        id_clave=claves_libres[id]
        mycursor.execute("SELECT id_juego, fecha_anexion FROM claves_juegos WHERE id_clave = %s", (id_clave))
        datos = mycursor.fetchone()
        #obtenemos el id del juego perteneciente a la clave
        id_juego = datos[0]
        #ponemos que el usuario compro dicha clave 4 semanas de su compra por ejemplo
        fecha = (datos[1])+dt.timedelta(weeks=+4)
        mycursor.execute("SELECT precio FROM juegos WHERE id_juego = %s", (id_juego,))
        preciojuego = mycursor.fetchone()
        mycursor.execute("SELECT porcentaje_a_aplicar,fecha_creacion,fecha_vencimiento FROM descuentos WHERE id_juego = %s", (id_juego,))
        descuentos = mycursor.fetchall()
        des=1
        #COMPROBAMOS AHORA SI EXISTE DESCUENTO PARA DICHO Juego
        for descuento in descuentos:
            fechaini=descuento[1]
            fechafin=descuento[2]
            des=descuento[0]
            if(fecha>fechaini and fecha<fechafin):
                break
       
        preciojuego = int(preciojuego[0] * (des/100))
        usuario=i
        c={}
        c["id_usuario"] = usuario
        usuario_db = mycol.find_one(c)
        biblioteca = {}
        update = {}
        biblioteca = usuario_db['biblioteca']
        if biblioteca == 0:
            biblioteca = []
        juego={}
        juego['id_juego'] = id_juego
        juego['valoraciones'] = random.randrange(1,11)
        juego['comentarios'] = random_text()
        biblioteca.append(juego)
        update["$set"] = {}
        update["$set"]['biblioteca'] = biblioteca
        a=mycol.update_one({ "id_usuario": usuario }, update)
        print (a.modified_count)
        if(a.modified_count==1):
            #vamos a actualizar la clave, modificando el estado a vendido
            mycursor.execute("UPDATE claves_juegos SET vendido = 1 WHERE id_clave = %s", (id_clave))
            #ahora añadimos la transacion
            sql = "INSERT INTO transacciones_venta (precio_total, fecha_venta) VALUES (%s, %s)"
            val = (preciojuego, fecha)
            mycursor.execute(sql,val)

            mycursor.execute("SELECT MAX(id_transaccion) FROM transacciones_venta")
            id_transac = mycursor.fetchone()[0]
            sql = "INSERT INTO venta_juegos (id_usuario, id_transaccion, id_claves_juego, precio) VALUES (%s, %s, %s, %s)"
            ran = random.randrange(0,len(claves_libres))
            id_clave = claves_libres[ran][0]
            #print((usuario, id_transac, id_clave, preciojuego))
            claves_libres.pop(ran)
            val = (usuario, id_transac, id_clave, preciojuego)
            mycursor.execute(sql,val)
            mydb.commit()
    
        

# Flujo de ventas de DLC sobre los usuarios anteriores

## Para cada usuario que tiene juegos se va generar un random de 0 o 1 para cada juego, si sale 1 procederá a comprar un dlc para ese juego si es que tiene

In [ ]:
import random
import mysql.connector
import json
import string
from pymongo import MongoClient
import datetime as dt

def random_string(num):
    return ''.join([random.choice(string.ascii_letters + string.digits) for n in range(num)]) + ' '

def random_text():
    num = random.randrange(1,15)
    num2 = random.randrange(1,200)
    ret = ''
    for i in range(0, num2) :
        ret.join(random_string(num))
    return ret
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["usuarios"]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)

mycursor = mydb.cursor(buffered=True)



for i in range(1,701):
    #procedemos a sacar los juegos que tiene el usuario
    usuario=i
    c["id_usuario"] = usuario
    usuario_db = mycol.find_one(c)
    #recorremos los juegos del usuario
    for jue in usuario_db["biblioteca"]:
        id_juego=jue["id_juego"]
        #compra=random.randrange(0,2)
        compra=1 #forzamos a que todos quieran comprar dlc si es que su juego tiene
        if(compra):
            #si el usuario quiere comprar dlc, selecionamos el id del dlc si es que tiene
            mycursor.execute("SELECT id_dlc,precio FROM dlcs WHERE id_juego = %s", (id_juego,))
            datos = mycursor.fetchone()
            #si existe dlc para ese juego procedemos a comprarlo
            if(datos):
                id_dlc=datos[0]
                precio=datos[1]
                #comprobamos que existan claves disponibles para ese DLC
                mycursor.execute("SELECT id_clave,fecha_anexion FROM claves_dlc  WHERE id_dlc = %s; ", (id_dlc,))
                dat=mycursor.fetchone()
                if(dat):
                    id_clave=dat[0]
                    fecha = (dat[1])+dt.timedelta(weeks=+4)
                    #ahora procedemos a comprobar si existe algun descuento para dicho dlc
                    mycursor.execute("SELECT porcentaje_a_aplicar,fecha_creacion,fecha_vencimiento FROM descuentos WHERE id_dlc = %s", (id_dlc,))
                    descuentos = mycursor.fetchall()
                    des=1
                    #COMPROBAMOS AHORA SI EXISTE DESCUENTO PARA DICHO DLC
                    for descuento in descuentos:
                        fechaini=descuento[1]
                        fechafin=descuento[2]
                        descuento=descuento[0]
                        if(fecha>fechaini and fecha<fechafin):
                            break
                    preciodlc = int(precio * (des/100))
                    
                    #creamos el formato para añadir el dlc dentro del juego correspondiente del usuario
                    dlc={}
                    dlc['id_dlc'] = id_dlc
                    dlc['valoraciones'] = random.randrange(1,11)
                    dlc['comentarios'] = random_text()
                    a=mycol.update_one({ "id_usuario": usuario,"biblioteca.id_juego":id_juego },{
                        '$push': {
                            'biblioteca.$.dlcs' : dlc
                        }
                    })
                    print (a.modified_count)
                    #si se ha añadido correctamente procedemos a insertar en transacciones y en ventas el movimiento
                    #también actualizamos la tabla de claves dlc, ya que la clave del dlc vendido pasa a estado vendido
                    if(a.modified_count==1):
                        sql = "INSERT INTO transacciones_venta (precio_total, fecha_venta) VALUES (%s, %s)"
                        val = (preciodlc, fecha)
                        mycursor.execute(sql,val)
    
                        #vamos a actualizar la clave, modificando el estado a vendido
                        mycursor.execute("UPDATE claves_dlc SET vendido = 1 WHERE id_clave = %s", (id_clave,))
                
                        mycursor.execute("SELECT MAX(id_transaccion) FROM transacciones_venta")
                        id_transac = mycursor.fetchone()[0]
                        sql = "INSERT INTO venta_dlcs (id_usuario, id_tansaccion, id_claves_dlc,precio  ) VALUES (%s, %s, %s, %s)"
                        print((usuario, id_transac, id_clave, preciodlc))
                        val = (usuario, id_transac, id_clave, preciodlc)
                        mycursor.execute(sql,val)
                        mydb.commit()

In [ ]:
import pandas as pd
from pymongo import MongoClient
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["usuarios"]

df =  pd.DataFrame(list(mycol.find()))
df.to_csv (r'usuarios.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path